# Loading and preparing data

In the previous lesson, we learned how to construct a chain. Now, we'll go over some techniques to store our own documents for later retrieval to ground the LLM's generation in our own context. This is generally called Retrieval Augmented Generation (or RAG).

The basic flow is as follows:

1. Load documents from a source.
2. Split the docs into chunks small enough to fit into an LLM's context window and avoid distraction.
3. Embed the chunks in a vectorstore to allow for later retrieval based on input queries.
4. Retrieval of relevant previously-split chunks.
5. Generating a final output with retrieved chunks as context.

![](./static/images/rag_diagram.png)

We'll cover the first two steps in this lesson.

## Loading

We'll need some source documents to start. LangChain includes many other document loaders for many sources of data.

For example, we can load code from a GitHub repo:

In [1]:
import "npm:dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
import { GithubRepoLoader } from "npm:langchain@0.0.178/document_loaders/web/github";
// Peer dependency, used to support .gitignore syntax
import ignore from "npm:ignore";

// Will not include anything under "ignorePaths"
const loader = new GithubRepoLoader(
  "https://github.com/langchain-ai/langchainjs",
  { recursive: false, ignorePaths: ["*.md", "yarn.lock"] }
);
const docs = await loader.load();
console.log({ docs });

{
  docs: [
    Document {
      pageContent: "# top-most EditorConfig file\n" +
        "root = true\n" +
        "\n" +
        "# Unix-style newlines with a newline ending every file\n" +
        "[*]"... 17 more characters,
      metadata: {
        source: ".editorconfig",
        repository: "https://github.com/langchain-ai/langchainjs",
        branch: "main"
      }
    },
    Document {
      pageContent: "* text=auto eol=lf",
      metadata: {
        source: ".gitattributes",
        repository: "https://github.com/langchain-ai/langchainjs",
        branch: "main"
      }
    },
    Document {
      pageContent: "node_modules/\n" +
        "dist/\n" +
        "dist-cjs/\n" +
        "lib/\n" +
        ".turbo\n" +
        ".eslintcache\n" +
        ".env\n" +
        ".env.local\n" +
        "yarn-error.log\n" +
        "docs/_dist/\n" +
        "\n" +
        "."... 340 more characters,
      metadata: {
        source: ".gitignore",
        repository: "https://github.com/

We can see that the resulting documents contain fields for `pageContent` and `metadata`.

We can also load from a PDF, such as [this transcript](https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture01.pdf) of Andrew Ng's famous CS229 course.

In [7]:
// Peer dependency
import * as parse from "npm:pdf-parse";
import { PDFLoader } from "npm:langchain@0.0.178/document_loaders/fs/pdf";

const loader = new PDFLoader("./static/docs/MachineLearning-Lecture01.pdf");

const rawCS229Docs = await loader.load();

rawCS229Docs

[
  Document {
    pageContent: "MachineLearning-Lecture01  \n" +
      "Instructor (Andrew Ng):\n" +
      " Okay. Good morning. Welcome to CS229, the machi"... 3042 more characters,
    metadata: {
      source: "./static/docs/MachineLearning-Lecture01.pdf",
      pdf: {
        version: "1.10.100",
        info: {
          PDFFormatVersion: "1.4",
          IsAcroFormPresent: false,
          IsXFAPresent: false,
          Title: "",
          Author: "",
          Creator: "PScript5.dll Version 5.2.2",
          Producer: "Acrobat Distiller 8.1.0 (Windows)",
          CreationDate: "D:20080711112523-07'00'",
          ModDate: "D:20080711112523-07'00'"
        },
        metadata: Metadata { _metadata: [Object: null prototype] },
        totalPages: 22
      },
      loc: { pageNumber: 1 }
    }
  },
  Document {
    pageContent: "many biologers are there here? Wow, just a \n" +
      "few, not many. I'm surprised. Anyone from \n" +
      "statistics? O"... 1024 more characters,
 

LangChain supports a wide range of other document loaders as well.

## Splitting

Because chunks are passed to the LLM after retrieval, our goal with splitting is to try to keep semantically related ideas together in the same chunk so that the LLM gets an entire retrieved idea.

There are many different strategies for splitting on different types of data. For example, we can use code-specific delimters, which are optimized to split input docs into JavaScript functions and classes for easier querying along those lines:

In [4]:
import { RecursiveCharacterTextSplitter } from "npm:langchain@0.0.178/text_splitter";

const splitter = RecursiveCharacterTextSplitter.fromLanguage("js", {
  chunkSize: 32,
  chunkOverlap: 0,
});
const code = `function helloWorld() {
console.log("Hello, World!");
}
// Call the function
helloWorld();`;

await splitter.splitText(code);

[
  "function helloWorld() {",
  'console.log("Hello, World!");\n}',
  "// Call the function",
  "helloWorld();"
]

Above, the splitter divides the function and function call along sensible boundaries smaller than the chunk size. If we had just split naively, we might get chunks containing half of a `console.log()` statement, which makes the LLM's job more difficult when generating from retrieved chunks.

LangChain includes several different options for different types of content, including Markdown and HTML. For generic written text, the `RecursiveCharacterTextSplitter` splits chunks on paragraphs, which is a good starting point.

In [5]:
const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 512,
  chunkOverlap: 64,
});

This will produce chunks with a maximum size of 512 characters and no overlap between chunks. There's no one-size-fits-all for these values - it will depend on your specific data source and the LLM you're using. Overlap can be advantageous to help keep semantic ideas in the same chunk, but too much can be inefficient.

Now, let's split our previously ingested raw docs from the CSS229 lesson transcript into chunks:

In [6]:
const splitDocs = await splitter.splitDocuments(rawCS229Docs);

splitDocs

[
  Document {
    pageContent: "MachineLearning-Lecture01  \n" +
      "Instructor (Andrew Ng):\n" +
      " Okay. Good morning. Welcome to CS229, the machi"... 404 more characters,
    metadata: {
      source: "./static/docs/MachineLearning-Lecture01.pdf",
      pdf: {
        version: "1.10.100",
        info: {
          PDFFormatVersion: "1.4",
          IsAcroFormPresent: false,
          IsXFAPresent: false,
          Title: "",
          Author: "",
          Creator: "PScript5.dll Version 5.2.2",
          Producer: "Acrobat Distiller 8.1.0 (Windows)",
          CreationDate: "D:20080711112523-07'00'",
          ModDate: "D:20080711112523-07'00'"
        },
        metadata: Metadata { _metadata: [Object: null prototype] },
        totalPages: 22
      },
      loc: { pageNumber: 1, lines: { from: 1, to: 13 } }
    }
  },
  Document {
    pageContent: "I actually think that machine learning is th\n" +
      "e most exciting field of all the computer \n" +
      "sciences. So"

Looks great! In the next lesson, we will show how to embed and add these chunks to a vectorstore.